## Before training

This program saves the last 3 generations of models to Google Drive. Since 1 generation of models is >1GB, you should have at least 3GB of free space in Google Drive. If you do not have such free space, it is recommended to create another Google Account.

Training requires >10GB VRAM. (T4 should be enough) Inference does not require such a lot of VRAM.

## Installation

In [1]:
#@title Check GPU
!nvidia-smi

Thu Jun  8 22:41:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
|  0%   45C    P8    18W / 125W |      6MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
#@title Install dependencies
#@markdown pip may fail to resolve dependencies and raise ERROR, but it can be ignored.
!python -m pip install -U pip wheel
%pip install -U ipython 

#@markdown Branch (for development)
BRANCH = "none" #@param {"type": "string"}
if BRANCH == "none":
    %pip install -U so-vits-svc-fork
else:
    %pip install -U git+https://github.com/34j/so-vits-svc-fork.git@{BRANCH}

  Using cached pip-23.1.2-py3-none-any.whl (2.1 MB)
  Using cached wheel-0.40.0-py3-none-any.whl (64 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.38.4
    Uninstalling wheel-0.38.4:
      Successfully uninstalled wheel-0.38.4
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
  Using cached ipython-8.12.2-py3-none-any.whl (797 kB)
  Attempting uninstall: ipython
    Found existing installation: ipython 8.12.0
    Uninstalling ipython-8.12.0:
      Successfully uninstalled ipython-8.12.0
Note: you may need to restart the kernel to use updated packages.
  Using cached so_vits_svc_fork-4.0.1-py3-none-any.whl (92 kB)
  Using cached soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl (1.2 MB)
  Using cached cm_time-0.1.2-py3-none-any.whl (7.8 kB)
  Using cached fastapi-0.88.0-py3-none-any.whl (55 kB)
  Using cached librosa-0.10.0.post2-py3-none-any.whl (253 kB)
  Usi

## Training

In [ ]:
#@title Make dataset directory
!mkdir -p "dataset_raw"

In [ ]:
#!rm -r "dataset_raw"
#!rm -r "dataset/44k"

In [ ]:
#@title Copy your dataset
#@markdown **We assume that your dataset is in your Google Drive's `so-vits-svc-fork/dataset/(speaker_name)` directory.**
DATASET_NAME = "kiritan" #@param {type: "string"}
!cp -R /content/drive/MyDrive/so-vits-svc-fork/dataset/{DATASET_NAME}/ -t "dataset_raw/"

In [ ]:
#@title Download dataset (Tsukuyomi-chan JVS)
#@markdown You can download this dataset if you don't have your own dataset.
#@markdown Make sure you agree to the license when using this dataset.
#@markdown https://tyc.rei-yumesaki.net/material/corpus/#toc6
# !wget https://tyc.rei-yumesaki.net/files/sozai-tyc-corpus1.zip
# !unzip sozai-tyc-corpus1.zip
# !mv "/content/つくよみちゃんコーパス Vol.1 声優統計コーパス（JVSコーパス準拠）/おまけ：WAV（+12dB増幅＆高音域削減）/WAV（+12dB増幅＆高音域削減）" "dataset_raw/tsukuyomi"

In [ ]:
#@title Automatic preprocessing
!svc pre-resample

In [ ]:
!svc pre-config

In [ ]:
#@title Copy configs file
!cp configs/44k/config.json drive/MyDrive/so-vits-svc-fork

In [ ]:
F0_METHOD = "dio" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]
!svc pre-hubert -fm {F0_METHOD}

In [ ]:
#@title Train
%load_ext tensorboard
%tensorboard --logdir drive/MyDrive/so-vits-svc-fork/logs/44k
!svc train --model-path drive/MyDrive/so-vits-svc-fork/logs/44k

## Training Cluster model

In [ ]:
!svc train-cluster --output-path drive/MyDrive/so-vits-svc-fork/logs/44k/kmeans.pt

## Inference

In [ ]:
#@title Get the author's voice as a source
import random
NAME = str(random.randint(1, 49))
TYPE = "fsd50k" #@param ["", "digit", "dog", "fsd50k"]
CUSTOM_FILEPATH = "" #@param {type: "string"}
if CUSTOM_FILEPATH != "":
    NAME = CUSTOM_FILEPATH
else:
    # it is extremely difficult to find a voice that can download from the internet directly
    if TYPE == "dog":
        !wget -N f"https://huggingface.co/datasets/437aewuh/dog-dataset/resolve/main/dogs/dogs_{NAME:.0000}.wav" -O {NAME}.wav
    elif TYPE == "digit":
        # george, jackson, lucas, nicolas, ...
        !wget -N f"https://github.com/Jakobovski/free-spoken-digit-dataset/raw/master/recordings/0_george_{NAME}.wav" -O {NAME}.wav
    elif TYPE == "fsd50k":
        !wget -N f"https://huggingface.co/datasets/Fhrozen/FSD50k/blob/main/clips/dev/{10000+int(NAME)}.wav" -O {NAME}.wav
    else:
        !wget -N f"https://zunko.jp/sozai/utau/voice_{"kiritan" if NAME < 25 else "itako"}{NAME % 5 + 1}.wav" -O {NAME}.wav
from IPython.display import Audio, display
display(Audio(f"{NAME}.wav"))

In [1]:
!svc -h

[04:35:55] INFO     [04:35:55] Version: 4.0.1                     ]8;id=898352;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=751327;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py#31\31]8;;\
Usage: svc [OPTIONS] COMMAND [ARGS]...

  so-vits-svc allows any folder structure for training data.
  However, the following folder structure is recommended.
      When training: dataset_raw/{speaker_name}/**/{wav_name}.{any_format}
      When inference: configs/44k/config.json, logs/44k/G_XXXX.pth
  If the folder structure is followed, you DO NOT NEED TO SPECIFY model path, config path, etc.
  (The latest model will be automatically loaded.)
  To train a model, run pre-resample, pre-config, pre-hubert, train.
  To infer a model, run infer.

Options:
  -h, --help  Show this message and exit.

Commands:
  clean          Clean up files, only useful if you are

In [2]:
!svc infer -h

[04:36:08] INFO     [04:36:08] Version: 4.0.1                     ]8;id=651193;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=197926;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py#31\31]8;;\
Usage: svc infer [OPTIONS] INPUT_PATH

  Inference

Options:
  -o, --output-path PATH          path to output dir
  -s, --speaker TEXT              speaker name
  -m, --model-path PATH           path to model  [default: logs/44k]
  -c, --config-path PATH          path to config  [default: configs/44k/config.json]
  -k, --cluster-model-path PATH   path to cluster model
  -re, --recursive                Search recursively
  -t, --transpose INTEGER         transpose  [default: 0]
  -db, --db-thresh INTEGER        threshold (DB) (RELATIVE)  [default: -20]
  -fm, --f0-method [crepe|crepe-tiny|parselmouth|dio|harvest]
                                  f0 predictio

## Preprocess target

In [1]:
!pip install AudioConverter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 5.6 MB/s eta 0:00:00
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3


In [5]:
import os, shutil
from os.path import dirname, abspath
from utils.demucs.utils import separate

# create output dir
filename = 'iu_strawberry_concert_preview'
output_dir = os.path.join('/home/arvin/so-vits-svc-fork/audio/', filename)
demucs_dir = os.path.join(output_dir, 'demucs')

# DEMUCS
separate(inp=output_dir, outp=demucs_dir)

Going to separate the files:
/home/arvin/so-vits-svc-fork/audio/iu_strawberry_concert_preview/iu_strawberry_concert_preview.mp3
With command:  python3 -m demucs.separate -o /home/arvin/so-vits-svc-fork/audio/iu_strawberry_concert_preview/demucs -n htdemucs --mp3 --mp3-bitrate=320
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/arvin/so-vits-svc-fork/audio/iu_strawberry_concert_preview/demucs/htdemucs
Separating track /home/arvin/so-vits-svc-fork/audio/iu_strawberry_concert_preview/iu_strawberry_concert_preview.mp3


100%|██████████████████████████████████████████████| 269.09999999999997/269.09999999999997 [00:10<00:00, 24.93seconds/s]


In [7]:
#@title Use trained model
#@markdown **Put your .wav file in `so-vits-svc-fork/audio` directory**
from IPython.display import Audio, display

PATH = f'/home/arvin/so-vits-svc-fork/audio/{filename}/demucs/htdemucs/{filename}'
VERSION = 109
NAME = 'vocals'

# !svc infer drive/MyDrive/so-vits-svc-fork/audio/{NAME}.wav -m drive/MyDrive/so-vits-svc-fork/logs/44k/ -c drive/MyDrive/so-vits-svc-fork/logs/44k/config.json
!svc infer -t -5 -fm crepe -na {PATH}/{NAME}.mp3 -m /home/arvin/so-vits-svc-fork/logs/44k/{VERSION} -c /home/arvin/so-vits-svc-fork/logs/44k/config.json

# display(Audio(f"drive/MyDrive/so-vits-svc-fork/audio/{NAME}.out.wav", autoplay=True))

[09:09:52] WARNING  [09:09:52] auto_predict_f0 = False,          ]8;id=136037;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=401644;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py#255\255]8;;\
                    transpose = -5. If you want to change the                   
                    pitch, please set transpose.Generally                       
                    transpose = 0 does not work because your                    
                    voice pitch and target voice pitch are                      
                    different.                                                  
           INFO     [09:09:52] Since model_path is a directory,  ]8;id=23697;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=274842;file:///home/arvin/miniconda3/envs/svc-demucs

In [1]:
#@title Use trained model
#@markdown **Put your .wav file in `so-vits-svc-fork/audio` directory**
from IPython.display import Audio, display

VERSION = 109
NAME = 'vocals'

# !svc infer drive/MyDrive/so-vits-svc-fork/audio/{NAME}.wav -m drive/MyDrive/so-vits-svc-fork/logs/44k/ -c drive/MyDrive/so-vits-svc-fork/logs/44k/config.json
!svc infer -t -5 -fm crepe -na /home/arvin/so-vits-svc-fork/audio/iu_blueming_orig_1.mp3 -m /home/arvin/so-vits-svc-fork/logs/44k/{VERSION} -c /home/arvin/so-vits-svc-fork/logs/44k/config.json

# display(Audio(f"drive/MyDrive/so-vits-svc-fork/audio/{NAME}.out.wav", autoplay=True))

[10:38:58] WARNING  [10:38:58] auto_predict_f0 = False,          ]8;id=560303;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=977168;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py#255\255]8;;\
                    transpose = -5. If you want to change the                   
                    pitch, please set transpose.Generally                       
                    transpose = 0 does not work because your                    
                    voice pitch and target voice pitch are                      
                    different.                                                  
           INFO     [10:38:58] Since model_path is a directory,  ]8;id=5594;file:///home/arvin/miniconda3/envs/svc-demucs/lib/python3.8/site-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=173907;file:///home/arvin/miniconda3/envs/svc-demucs/

In [ ]:
##@title Use trained model (with cluster)
!svc infer {NAME}.wav -s speaker -r 0.1 -m drive/MyDrive/so-vits-svc-fork/logs/44k/ -c drive/MyDrive/so-vits-svc-fork/logs/44k/config.json -k drive/MyDrive/so-vits-svc-fork/logs/44k/kmeans.pt
display(Audio(f"{NAME}.out.wav", autoplay=True))

### Pretrained models

In [ ]:
#@title https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/tree/main
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/G_riri_220.pth"
!wget -N "https://huggingface.co/TachibanaKimika/so-vits-svc-4.0-models/resolve/main/riri/config.json"

In [ ]:
!svc infer {NAME}.wav -c config.json -m G_riri_220.pth
display(Audio(f"{NAME}.out.wav", autoplay=True))

In [ ]:
#@title https://huggingface.co/therealvul/so-vits-svc-4.0/tree/main
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/G_166400.pth"
!wget -N "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/config.json"

In [ ]:
!svc infer {NAME}.wav --speaker "Pinkie {neutral}" -c config.json -m G_166400.pth
display(Audio(f"{NAME}.out.wav", autoplay=True))